In [3]:
# Sorted Class List
import os

have_ids = []

for dir in os.listdir('train/images'):
    have_ids.append(str(dir))
    
data = open('words.txt','r').read()

classes = {}
for line in data.split("\n"):    
    imagenet_id, imagenet_desc = line.split("\t")
    classes[imagenet_id] = imagenet_desc
    
have_classes = {}

for imagenet_id in have_ids:
    have_classes[imagenet_id] = classes[imagenet_id]
    
have_classes_list = []
for imagenet_id in have_classes:
    have_classes_list.append(have_classes[imagenet_id].lower())
    
have_classes_list = sorted(have_classes_list)

output = ""
for desc in have_classes_list:
    output += desc + "\n"

open('sorted_classes.txt', 'w').write(output)

21675

In [2]:
# Convert from ImageNet XML to DIGITS format

import xmltodict
import os
import collections

def imagenet_to_digits_detect(source, dest, classes):
    have_ids = []

    for dir in os.listdir(source):
        have_ids.append(str(dir))

    class_xml = {}

    for imagenet_id in have_ids:
        xml_files = ["%s/%s/%s" % (source, imagenet_id, str(x)) 
                     for x in os.listdir('%s/%s' % (source, imagenet_id))]
        for xml_file in xml_files:
            doc = xmltodict.parse(open(xml_file, 'r').read())['annotation']
            
            file_name = doc['filename']
            
            objects = doc['object']
            
            if(type(objects) == collections.OrderedDict):
                objects = [objects]
            
            digits_str = ""
            for object in objects:

                name = object['name']

                xmin = int(object['bndbox']['xmin'])
                xmax = int(object['bndbox']['xmax'])
                ymin = int(object['bndbox']['ymin'])
                ymax = int(object['bndbox']['ymax'])
                
                truncated = float(object['truncated'])

                digits_str += "%s %.1f 0 0.0 %.2f %.2f %.2f %.2f 0.0 0.0 0.0 0.0 0.0 0.0 0.0\n" % (name, truncated, xmin, ymin, xmax, ymax)
            
            digits_str = digits_str.strip()
            
            try:
                os.mkdir('%s/%s' % (dest, imagenet_id))
            except FileExistsError:
                pass

            open('%s/%s/%s.txt' % (dest, imagenet_id, file_name), 'w').write(digits_str)
            
data = open('words.txt','r').read()

classes = {}
for line in data.split("\n"):    
    imagenet_id, imagenet_desc = line.split("\t")
    classes[imagenet_id] = imagenet_desc
    
imagenet_to_digits_detect("imagenet_xml/val", "val", classes)
imagenet_to_digits_detect("imagenet_xml/train", "train/labels", classes)


In [1]:
import os
from shutil import copy
import struct
from PIL import Image


# Create Dataset
dataset_name = "squirrel"
classes = ['n02356798']

max_width = 1248
max_height = 384

os.mkdir("datasets/%s" % dataset_name)
os.mkdir("datasets/%s/train" % dataset_name)
os.mkdir("datasets/%s/train/labels" % dataset_name)
os.mkdir("datasets/%s/train/images" % dataset_name)

os.mkdir("datasets/%s/val" % dataset_name)
os.mkdir("datasets/%s/val/labels" % dataset_name)
os.mkdir("datasets/%s/val/images" % dataset_name)

def image_size(path):
    return Image.open(path).size

for wnid in classes:
    
    print("Processing %s" % wnid)
    labels = os.listdir("train/labels/%s" % wnid)
    
    for label in labels:
        name, ext = label.split(".")
        
        image = "%s.JPEG" % name
        
        width, height = image_size("train/images/%s/%s" % (wnid, image))
        
        if width >= max_width or height >= max_height:
            print("Skipping %s %dx%d" % (image, width, height))
            continue
        
        copy("train/labels/%s/%s" % (wnid, label), "datasets/%s/train/labels/%s" % (dataset_name, label))
        copy("train/images/%s/%s" % (wnid, image), "datasets/%s/train/images/%s" % (dataset_name, image))

for label in os.listdir('val/labels'):
        name, ext = label.split(".")
        
        image = "%s.JPEG" % name
        
        labeldata = open("val/labels/%s" % (label), 'r').read()
        
        have = False
        for wnid in classes:
            if labeldata.find(wnid) != -1:
                have = True
                break
        
        if not have:
            continue
        
        width, height = image_size("val/images/%s" % (image))
        
        if width >= max_width or height >= max_height:
            print("Skipping %s %dx%d" % (image, width, height))
            continue

        copy("val/labels/%s" % (label), "datasets/%s/val/labels/%s" % (dataset_name, label))
        copy("val/images/%s" % (image), "datasets/%s/val/images/%s" % (dataset_name, image))



Processing n02356798
Skipping n02356798_5175.JPEG 541x393
Skipping n02356798_2464.JPEG 375x500
Skipping n02356798_1661.JPEG 460x500
Skipping n02356798_7127.JPEG 640x480
Skipping n02356798_1499.JPEG 374x399
Skipping n02356798_818.JPEG 500x407
Skipping n02356798_760.JPEG 306x432
Skipping n02356798_1073.JPEG 500x400
Skipping n02356798_3255.JPEG 940x705
Skipping n02356798_531.JPEG 334x500
Skipping n02356798_1826.JPEG 334x500
Skipping n02356798_1972.JPEG 433x500
Skipping n02356798_6618.JPEG 400x590
Skipping n02356798_2220.JPEG 347x500
Skipping n02356798_5886.JPEG 500x443
Skipping n02356798_4430.JPEG 375x500
Skipping n02356798_3669.JPEG 600x701
Skipping n02356798_6780.JPEG 496x500
Skipping n02356798_6210.JPEG 800x600
Skipping n02356798_4423.JPEG 369x500
Skipping n02356798_4954.JPEG 333x500
Skipping n02356798_2554.JPEG 500x400
Skipping n02356798_4225.JPEG 720x480
Skipping n02356798_3673.JPEG 292x432
Skipping n02356798_3771.JPEG 498x500
Skipping n02356798_4750.JPEG 540x720
Skipping n02356798_4